<a href="https://colab.research.google.com/github/AwkwaBear/EE-628/blob/main/sg13g2_LDO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Design and Optimization of Analog LDO Using sg13g2 PDK with Relational Graph Neural Network and Reinforcement Learning


|Name|
|:--:|
|Vikas Shukla |
|Zion McDowell|
|Jie Zhao|
|Anthony Gasbarro|


### Install dependencies

## First we follow a modified version of the [Dockerfile](https://github.com/iic-jku/IIC-OSIC-TOOLS/blob/main/_build/Dockerfile) build Script from the [IIC-OSIC-Tools Github Page](https://github.com/iic-jku/IIC-OSIC-TOOLS/tree/main)

This script is adapted one section at a time to the Google Colab environment's syntax

In [5]:
import os

#######################################################################
# Setup base image
#######################################################################

# Set environment variables
os.environ['DEBIAN_FRONTEND'] = 'noninteractive'
os.environ['TZ'] = 'Pacific/Honolulu'
os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'
os.environ['TOOLS'] = '/foss/tools'
os.environ['PDK_ROOT'] = '/foss/pdks'
os.environ['DESIGNS'] = '/foss/designs'
os.environ['EXAMPLES'] = '/foss/examples'

# Create directories
!mkdir -p "$TOOLS" "$PDK_ROOT" "$DESIGNS" "$EXAMPLES"

#######################################################################
# Add base packages (install via pip, npm, or gem)
#######################################################################


- sg13g2 [Process Development Kit]
- Ngspice [Circuit Simulation Tool]
- Conda [Python Development Virtual Environment]
- Pytorch [Machine Learning Framework]
- PyG [Graph Neural Network Library]
- Tabulate [Data Table Export Interface Library]

Note: that after the installation of these packages, **you may need to restart the runtime to use these newly installed packages**.

In [1]:
import os

# install conda.
CONDA_PREFIX = os.environ.get('CONDA_PREFIX', None)
if not CONDA_PREFIX:
  !python -m pip install condacolab
  import condacolab
  condacolab.install()



✨🍰✨ Everything looks OK!


## Install PDK and Simulation Tools

### Install OpenVaf (Verilog-A Compiler)

In [4]:
# Define necessary variables
OPENVAF_REPO_URL = "https://github.com/path/to/openvaf/repo.git"
OPENVAF_REPO_COMMIT = "commit_hash_here"
OPENVAF_NAME = "openvaf"
TOOLS = "/content/tools"  # Example tools directory

import os

# Ensure cargo and necessary Rust components are installed
# This step assumes LLVM 15 or 16 is already installed. If not, you might need to install it.
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
!source $HOME/.cargo/env

# Clone the repository with the specified commit
REPO_COMMIT_SHORT = OPENVAF_REPO_COMMIT[:7]

!git clone --filter=blob:none "{OPENVAF_REPO_URL}" "{OPENVAF_NAME}"
%cd "{OPENVAF_NAME}"
!git checkout "{OPENVAF_REPO_COMMIT}"

# Apply the patch
patch_content = """
diff --git a/openvaf/target/build.rs b/openvaf/target/build.rs
index 13db6e5..dada68e 100644
--- a/openvaf/target/build.rs
+++ b/openvaf/target/build.rs
@@ -36,7 +36,7 @@ fn gen_msvcrt_importlib(sh: &Shell, arch: &str, target: &str, check: bool) {
     let ucrt_obj = out_dir.join(format!("ucrt_{arch}.obj"));
     cmd!(
         sh,
-        "clang-cl /c /Zl /GS- /clang:--target={target}-pc-windows-msvc /clang:-o{ucrt_obj} {ucrt_src}"
+        "clang -c -fno-autolink -fno-stack-protector --target={target}-pc-windows-msvc -o{ucrt_obj} {ucrt_src}"
     )
     .run()
     .expect("ucrt compilation succeeds");
"""

!echo "{patch_content}" > /tmp/openvaf_patch.diff
!patch -p1 < /tmp/openvaf_patch.diff

# Build using cargo
!cargo build --release --bin openvaf

# Copy the built binary to the specified location
target_dir = f"{TOOLS}/{OPENVAF_NAME}/{REPO_COMMIT_SHORT}/bin"
!mkdir -p "{target_dir}"
!cp target/release/openvaf "{target_dir}"

curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
info: downloading installer
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
sh: 172: cannot

In [2]:
# install sg13g2
import os

# Assuming PDK_ROOT and TOOLS are already set in your environment, if not, set them here
os.environ['PDK_ROOT'] = '/content/foss/pdks'  # Example path, adjust as needed
os.environ['TOOLS'] = '/content/tools'  # Example path, adjust as needed
os.environ['OPENVAF_NAME'] = 'openvaf'  # Adjust according to your setup

# Install IHP-SG13G2
!mkdir -p "$PDK_ROOT"

IHP_PDK="ihp-sg13g2"
MY_PDK="sg13g2"

# Cloning the IHP Open PDK
!cd /tmp && git clone https://github.com/IHP-GmbH/IHP-Open-PDK.git ihp
%cd /tmp/ihp
!git checkout dev

# Modifying .sch files
!find . -name "*.sch" -exec sed -i '/pre_osdi/d' {} \;

# Moving the PDK to the desired location
if os.path.isdir(f"/tmp/ihp/{IHP_PDK}"):
  !mv /tmp/ihp/{IHP_PDK} "$PDK_ROOT/$MY_PDK"

# Compile .va models
%cd "$PDK_ROOT/$MY_PDK/libs.tech/ngspice/openvaf"
OPENVAF_VERSION = os.listdir(f"{os.environ['TOOLS']}/{os.environ['OPENVAF_NAME']}")[0]
!{os.environ['TOOLS']}/{os.environ['OPENVAF_NAME']}/{OPENVAF_VERSION}/bin/openvaf psp103_nqs.va

# install ngspice
# !conda install -c conda-forge ngspice

Cloning into 'ihp'...
remote: Enumerating objects: 4854, done.
remote: Counting objects: 100% (1272/1272), done.
remote: Compressing objects: 100% (538/538), done.
remote: Total 4854 (delta 745), reused 1137 (delta 695), pack-reused 3582
Receiving objects: 100% (4854/4854), 315.08 MiB | 19.66 MiB/s, done.
Resolving deltas: 100% (2118/2118), done.
Updating files: 100% (3788/3788), done.
/tmp/ihp
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'
mv: cannot stat '/tmp/ihp/{IHP_PDK}': No such file or directory
[Errno 2] No such file or directory: '$PDK_ROOT/$MY_PDK/libs.tech/ngspice/openvaf'
/tmp/ihp


FileNotFoundError: [Errno 2] No such file or directory: '/content/tools/openvaf'

## Install Other Python Tools

In [ ]:


# Install Pytorch with cpu-only option.
!conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 cpuonly -c pytorch

# Check the version of Python (should be larger than 3.7) and Pytorch.
import torch
import sys
print("Python version")
print (sys.version)

!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

# Install PyG to enable graph neural networks. We need to manually guide Colab to install the correct version of PyG based on which Pytorch version we have installed
# (https://colab.research.google.com/drive/17VFsRZeZa7rgtrQ39-RMiYIE0ePrSlgN?authuser=1#scrollTo=rFwf_0DrvXLv). In this case it is 1.13.1 with cpu-only option.
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-geometric

# Check the version of PyG.
import torch_geometric
print(torch_geometric.__version__)

# Install Gymnasium for the RL environment.
!pip install gymnasium

# Check Gymnasium version.
import gymnasium as gym
print(gym.__version__)

# Install tabulate for showing simulation results nicely during the runtime later.
!pip install tabulate

# Similarly, check tabulate version.
import tabulate
print(tabulate.__version__)

### Setting up g13g2 circuit simulation environment


Create some directory to organize files.

In [ ]:
!mkdir sg13g2_ldo
!mkdir sg13g2_ldo/simulations
!mkdir sg13g2_ldo/saved_weights
!mkdir sg13g2_ldo/saved_memories

Setting up Ngspice initialization for g13g2. Also do use 8 threads to speed up simulations.

In [ ]:
%%writefile /content/g13g2_ldo/simulations/.spiceinit
* ngspice initialization for g13g2
* assert BSIM compatibility mode with "nf" vs. "W"
set ngbehavior=hsa
* "nomodcheck" speeds up loading time
set ng_nomodcheck
set num_threads=8

Writing /content/g13g2_ldo/simulations/.spiceinit


Write LDO netlist. Noticed that it calls the analysis and variables of the netlist from another two spice scripts (`ldo_tb_analysis.spice` and `ldo_tb_vars.spice`). We will include them later.



In [ ]:
!conda install -c litex-hub open_pdks.sg13g2

/ - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
Solvi

In [ ]:
#TODO: MODIFY BELOW CODE FOR G13g2
%%writefile /content/sky130_ldo/simulations/ldo_tb.spice
** sch_path: /fs1/eecg/tcc/lizongh2/sky130_ldo/xschem/ldo_tb.sch
**.subckt ldo_tb Vreg Vreg1
*.opin Vreg
*.opin Vreg1
Vref Vref GND Vref
.save i(vref)
Vb net1 GND Vb
.save i(vb)
IL Vreg net2 dc IL PULSE(10u IL 0 10n 10n 50u 100u 0)
Vdd Vdd GND ac 1 dc Vdd
.save i(vdd)
x1 Vdd Vreg Vref net1 GND Vreg ldo
XCL Vreg GND sky130_fd_pr__cap_mim_m3_1 W=30 L=30 MF=M_CL m=M_CL
Rdummy net2 GND 1 m=1
Vref1 Vref1 GND Vref
.save i(vref1)
Vb1 net3 GND Vb
.save i(vb1)
IL1 Vreg1 GND IL
Vdd1 Vdd1 GND Vdd
.save i(vdd1)
x2 Vdd1 net4 Vref1 net3 GND Vreg1 ldo
XCL1 Vreg1 GND sky130_fd_pr__cap_mim_m3_1 W=30 L=30 MF=M_CL m=M_CL
Vprobe2 probe net4 dc 0
.save i(vprobe2)
Vprobe1 probe Vreg1 dc 0 ac 1
.save i(vprobe1)
Iprobe1 GND probe dc 0 ac 0
**** begin user architecture code

* Not loading the entire design corner which takes too long
*.lib /usr/local/share/pdk/sky130A/libs.tech/ngspice/sky130.lib.spice tt

* Instead just load tt corner which is faster
.param mc_mm_switch=0
.param mc_pr_switch=0
.include /usr/local/share/pdk/sky130A/libs.tech/ngspice/corners/tt.spice
.include /usr/local/share/pdk/sky130A/libs.tech/ngspice/r+c/res_typical__cap_typical.spice
.include /usr/local/share/pdk/sky130A/libs.tech/ngspice/r+c/res_typical__cap_typical__lin.spice
.include /usr/local/share/pdk/sky130A/libs.tech/ngspice/corners/tt/specialized_cells.spice

.control
* save all voltage and current
save all
.options savecurrents
set filetype=ascii
set units=degrees

.include /content/sky130_ldo/simulations/ldo_tb_analysis.spice
.endc

.include /content/sky130_ldo/simulations/ldo_tb_vars.spice

**** end user architecture code
**.ends

* expanding   symbol:  ldo.sym # of pins=6
** sym_path: /fs1/eecg/tcc/lizongh2/sky130_ldo/xschem/ldo.sym
** sch_path: /fs1/eecg/tcc/lizongh2/sky130_ldo/xschem/ldo.sch
.subckt ldo Vdd Vfb Vref Vb Vss Vreg
*.iopin Vb
*.iopin Vss
*.opin Vreg
*.iopin Vref
*.ipin Vdd
*.iopin Vfb
x1 Vdd net1 Vfb Vref Vb Vss diff_pair
XM6 Vreg net1 Vdd Vdd sky130_fd_pr__pfet_g5v0d10v5 L=L_pass W=W_pass nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=M_pass m=M_pass
XCfb net2 Vreg sky130_fd_pr__cap_mim_m3_1 W=10 L=10 MF=M_Cfb m=M_Cfb
XRfb net2 net1 Vss sky130_fd_pr__res_high_po_0p35 L=3 mult=M_Rfb m=M_Rfb
.ends


* expanding   symbol:  diff_pair.sym # of pins=6
** sym_path: /fs1/eecg/tcc/lizongh2/sky130_ldo/xschem/diff_pair.sym
** sch_path: /fs1/eecg/tcc/lizongh2/sky130_ldo/xschem/diff_pair.sch
.subckt diff_pair Vdd vout vinp vinm Vb Vss
*.iopin Vdd
*.ipin vinm
*.ipin vinp
*.iopin Vb
*.opin vout
*.iopin Vss
XM1 net1 vinp net2 Vss sky130_fd_pr__nfet_g5v0d10v5 L=L_M1 W=W_M1 nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM2 vout vinm net2 Vss sky130_fd_pr__nfet_g5v0d10v5 L=L_M2 W=W_M2 nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM3 vout net1 Vdd Vdd sky130_fd_pr__pfet_g5v0d10v5 L=L_M3 W=W_M3 nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM4 net1 net1 Vdd Vdd sky130_fd_pr__pfet_g5v0d10v5 L=L_M4 W=W_M4 nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM5 net2 Vb Vss Vss sky130_fd_pr__nfet_g5v0d10v5 L=L_M5 W=W_M5 nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
.ends

.GLOBAL GND
.end


Writing /content/sky130_ldo/simulations/ldo_tb.spice


FileNotFoundError: [Errno 2] No such file or directory: '/content/sky130_ldo/simulations/ldo_tb.spice'